In [ ]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn import metrics
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import (
    Ridge,RidgeCV,
    Lasso,LassoCV,
    ElasticNet, ElasticNetCV,
    LinearRegression
)
from sklearn.model_selection import(
    cross_val_score,
    train_test_split
)
from sklearn.preprocessing import (
    StandardScaler,
    PolynomialFeatures
)
from sklearn.feature_selection import SelectFromModel
pd.set_option('display.max_rows', 1000)  # or 1000
pd.set_option('display.max_columns', 1000)  # or 1000

from itertools import combinations
from itertools import permutations

from sklearn.preprocessing import OneHotEncoder

In [ ]:
df = pd.read_csv('datasets/train.csv')

In [ ]:
df.head(10)

In [ ]:
# rename column names 

new_columns = {
    'Id': 'id',
    'PID':'pid',
    'MS SubClass': 'ms_subclass',
    'MS Zoning': 'ms_zoning',
    'Lot Frontage': 'lot_front',
    'Lot Area': 'lot_area',
    'Street': 'street',
    'Alley':'alley',
    'Lot Shape': 'lot_shape',
    'Land Contour':'land_contour',
    'Utilities': 'util',
    'Lot Config': 'lot_config',
    'Land Slope': 'land_slope',
    'Neighborhood': 'neighborhood',
    'Condition 1': 'cond_1',    
    'Condition 2': 'cond_2',
    'Bldg Type': 'bldg_type',
    'House Style': 'house_style',
    'Overall Qual': 'overall_qual',
    'Overall Cond': 'overall_cond',
    'Year Built': 'year_built',
    'Year Remod/Add': 'year_remod_add',
    'Roof Style': 'roof_style',
    'Roof Matl': 'roof_matl',
    'Exterior 1st': 'ext_1st',
    'Exterior 2nd': 'ext_2nd',
    'Mas Vnr Type': 'mas_vnr_type',
    'Mas Vnr Area': 'mas_vnr_area',
    'Exter Qual': 'exter_qual',
    'Exter Cond': 'exter_cond',
    'Foundation': 'foundation',
    'Bsmt Qual': 'bsmt_qual',
    'Bsmt Cond': 'bsmt_cond',
    'Bsmt Exposure': 'bsmt_exposure',
    'BsmtFin Type 1': 'bsmtfin_type1',
    'BsmtFin SF 1': 'bsmtfin_sf1',
    'BsmtFin Type 2': 'bsmt_type2',
    'BsmtFin SF 2': 'bsmt_sf2',
    'Bsmt Unf SF': 'bsmt_unf_sf',
    'Total Bsmt SF': 'total_bsmt_sf',
    'Heating': 'heating',
    'Heating QC': 'heating_qc',
    'Central Air': 'central_air',
    'Electrical': 'electrical',
    '1st Flr SF': '1st_flr_sf',
    '2nd Flr SF': '2nd_flr_sf',
    'Low Qual Fin SF': 'low_qual_fin_sf',
    'Gr Liv Area': 'gr_liv_area',
    'Bsmt Full Bath': 'bsmt_full_bath',    
    'Bsmt Half Bath': 'bsmt_half_bath',
    'Full Bath': 'full_bath',
    'Half Bath': 'half_bath',
    'Bedroom AbvGr': 'bedroom_abv_gr',
    'Kitchen AbvGr': 'kitchen_abv_gr',
    'Kitchen Qual': 'kitchen_qual',
    'TotRms AbvGrd': 'tot_rms_abv_grd',
    'Functional': 'functional',
    'Fireplaces': 'fireplace',
    'Fireplace Qu': 'fireplace_qu',
    'Garage Type': 'garage_type',
    'Garage Yr Blt': 'garage_yr_blt',
    'Garage Finish': 'garage_finish',
    'Garage Cars': 'garag_cars',
    'Garage Area': 'garage_area',
    'Garage Qual': 'garage_qual',
    'Garage Cond': 'garage_cond',
    'Paved Drive': 'paved_drive',
    'Wood Deck SF': 'wood_deck_sf',
    'Open Porch SF': 'open_porch_sf',
    'Enclosed Porch': 'enclosed_porch',
    '3Ssn Porch': '3ssn_porch',
    'Screen Porch': 'screen_porch',
    'Pool Area': 'pool_area',
    'Pool QC': 'pool_qc',
    'Fence': 'fence',
    'Misc Feature': 'misc_feature',
    'Misc Val': 'misc_val',
    'Mo Sold': 'mo_sold',
    'Yr Sold': 'yr_sold',
    'Sale Type': 'sale_type',
    'SalePrice':'sale_price',
}

df.rename(columns=new_columns, inplace=True)

In [ ]:
df.head()

## EDA
- **Read the data dictionary.**
- Determine _what_ missing values mean.
- Figure out what each categorical value represents.
- Identify outliers.
- Consider whether discrete values are better represented as categorical or continuous. (Are relationships to the target linear?)

## Data Cleaning
- Decide how to impute null values.
- Decide how to handle outliers.
- Do you want to combine any features?
- Do you want to have interaction terms?
- Do you want to manually drop collinear features?

**Clean missing values**  

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
# build list of features with na 

feature_nan = [feature for feature in df.columns if df[feature].isnull().sum()]
for feat in feature_nan:
    print(feat, f'has {round((df[feat].isnull().mean())*100,2)}% missing values')

**Some missing values are bigger percentage than the other** 

- we need to find the correlation between the missing values and the sales price to judge if its good to remove or to keep

**Approach**

1) Larger values - might remove

2) Smaller values - need to conduct EDA and decide later

In [ ]:
# Lot_font has normal distrubtion centered across mean of 69

sns.histplot(x = 'lot_front', data =df)
print(df['lot_front'].mean())
print(df['lot_front'].median())

In [ ]:
# get mean and fill into NA
# why mean -->> because normally distributed centered at mean 69. 
df['lot_front'].fillna(df['lot_front'].mean(),inplace = True)

In [ ]:
# Lot_font has normal distrubtion centered across mean of 69

sns.histplot(x = 'lot_front', data =df)
print(df['lot_front'].mean())

In [ ]:
# change 'nan' to NoAlleyAccess
df['alley'] = df['alley'].map(lambda x: 'NoAlley' if pd.isnull(x) else x)

In [ ]:
# no meaning to the sales 
# drop alley

sns.boxplot(
    data = df,
    x = 'alley',
    y = 'sale_price'
)

Boxplot shows strong outliers in NoAlley, hence if need to use this data, has to clear outliers first. 

In [ ]:
# mas_vnr_type -->> categorical, insert most common type 
df.groupby(by='mas_vnr_type').count()['id']

mas_vnr_type already has a None column. Will need to classify nan into the mode of the types for minimal impact


In [ ]:
# fill missing values with mode of mas_vnr_type
df['mas_vnr_type'].fillna(df['mas_vnr_type'].mode()[0], inplace = True)

In [ ]:
# checked categories for mas_vnr_area
# found if mas vnr area == 0, then belongs to none
sns.histplot(x = 'mas_vnr_area', hue = 'mas_vnr_type', data =df)
plt.xlim(0,600)

In [ ]:
# checking nominal category: mas_vnr_type against sale price

plt.figure(figsize=(20,10))
sns.boxplot(
    x = 'mas_vnr_type',
    y = 'sale_price',
    data = df
)

In [ ]:
# create dictionary that stores mean of
mas_dict_mean = {key:val for key, val in df.groupby(['mas_vnr_type'])['mas_vnr_area'].mean().iteritems()}

mas_dict_mean

In [ ]:
# create function that returns mean based on mas_dict_mean if column has null values 
def enter_area(col):
    area = col[0]
    mas_type = col[1]
    if pd.isnull(area):
        return mas_dict_mean[mas_type]
    else:
        return area

In [ ]:
# fill in null values for -->> mas_vnr_area
df['mas_vnr_area'] = df[['mas_vnr_area', 'mas_vnr_type']].apply(enter_area, axis = 1)

In [ ]:
# missing values seems common across about bsmt columns 
df[df['bsmt_exposure'].isnull()]

In [ ]:
# create pairplot to check relationship between all bsmt parameters against sale_price
# noticed only total_basement_sf area has high correlation with sale_price
# checked only 58 out of 2051 data has NAN -->> meaning no basement 
# will want to change all null to 'NoBsmt'



plt.figure(figsize=(15,15))
sub_bsmt = ['bsmt_qual', 'bsmt_cond', 'bsmt_exposure',
       'bsmtfin_type1', 'bsmtfin_sf1', 'bsmt_type2', 'bsmt_sf2', 'bsmt_unf_sf',
       'total_bsmt_sf', 'sale_price']

sns.pairplot(
    df,
    x_vars = sub_bsmt,
    y_vars = ['sale_price'],
    
)

**EDA on basement null values**

bsmt_qual            55 null values

bsmt_cond            55 null values

bsmt_exposure        58 null values

bsmtfin_type1        55 null values

bsmtfin_sf1           1 null values

bsmt_type2           56 null values

In [ ]:
# Checked most of the categories have null values as there is no basement. 

df.loc[df['bsmt_exposure'].isnull(), ['bsmt_qual', 'bsmt_cond', 'bsmt_exposure',
       'bsmtfin_type1', 'bsmtfin_sf1', 'bsmt_type2', 'bsmt_sf2', 'bsmt_unf_sf',
       'total_bsmt_sf']]

In [ ]:
# Create none category for those without basement
df['bsmt_cond'].fillna('NoBsmt',inplace = True)

In [ ]:
# cross check category, makes sense that those without basement will have lowest mean sale price 
sns.boxplot(
    x = 'bsmt_cond',
    y = 'sale_price',
    data = df,
    order = ['Ex','Gd', 'TA', 'Fa', 'Po', 'NoBsmt']
)

In [ ]:
# Create none category for those without basement
df['bsmt_type2'].fillna('NoBsmt',inplace = True)

In [ ]:
# notice higher prices are under unfinised. 
# no basement is the lowest mean score, which fits the ordinal flow. 

sns.boxplot(
    x = 'bsmt_type2',
    y = 'sale_price',
    data = df,
    order = ['GLQ', 'ALQ','BLQ','Rec','LwQ','Unf','NoBsmt']
)

In [ ]:
# since it is numerical cat, need to take a look at mean. 
# found those with null, average sale price is lower than all the other cats 
# hence safe to create another cat for None. 

bsmt_mean_null = df.loc[df['bsmt_exposure'].isnull(), 'sale_price'].mean()
bsmt_mean = df.groupby('bsmt_exposure').mean()['sale_price']
print(f'mean of null values: {bsmt_mean_null}')
print('\n')
print(f'Mean values of basement exposure: {bsmt_mean}')
df['bsmt_exposure'].value_counts()

In [ ]:
# Create none category for those without basement
df['bsmt_exposure'].fillna('NoBsmt',inplace = True)

In [ ]:
# cross checked ordinally correct for No basement cat
sns.boxplot(
    x = 'bsmt_exposure',
    y = 'sale_price',
    data = df,
    order = ['Gd','Av','Mn','No','NoBsmt']
)

In [ ]:
# fill up null values with NoBsmt
df['bsmt_qual'].fillna('NoBsmt', inplace = True)

In [ ]:
# cross checked ordinally correct for No basement cat
sns.boxplot(
    x = 'bsmt_qual',
    y = 'sale_price',
    data = df,
    order = ['Ex','Gd', 'TA', 'Fa', 'Po', 'NoBsmt']
)

**Seeing small points for Po** 
- might need to remove it if want to use this variable as it might skew predicted data inaccurately. 


In [ ]:
# fill up null values with NoBsmt
df['bsmtfin_type1'].fillna('NoBsmt', inplace = True) 

In [ ]:
# cross checked ordinally correct for No basement cat

sns.boxplot(
    x = 'bsmtfin_type1',
    y = 'sale_price',
    data = df,
    order = ['GLQ', 'ALQ','BLQ','Rec','LwQ','Unf','NoBsmt']
)

In [ ]:
# checked row has no basement
df[df['total_bsmt_sf'].isnull()]

In [ ]:
# replace na with 0 feet square

df['bsmtfin_sf1'].fillna(0, inplace = True) 
df['bsmt_sf2'].fillna(0, inplace = True) 
df['total_bsmt_sf'].fillna(0, inplace = True) 
df['bsmt_unf_sf'].fillna(0, inplace = True) 

In [ ]:
sns.boxplot(
    x = 'fireplace_qu',
    y = 'sale_price',
    data = df,
    order = ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
)

In [ ]:
# Establish size of figure.
plt.figure(figsize = (10,5))


sns.boxplot(
    x = 'fireplace',
    y = 'sale_price',
    data = df,
#     hue = 'fireplace_qu',
#     hue_order = ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
    width = 0.6
);


In [ ]:
df.groupby('fireplace')['fireplace'].count()

**Observed rating 4 having only one point, might need to remove this variable if need to use as results might be skewed**

In [ ]:
# null values contain sale price mean close to 144000
df[df['fireplace_qu'].isnull()].agg(['mean', 'median'])['sale_price']

In [ ]:
# found that cat Po suites best for null values to be included without affecting much mean 
df.groupby(['fireplace_qu']).agg(['mean', 'median', 'count'])['sale_price']

In [ ]:
# fill na with NoFireplace
df['fireplace_qu'] = df['fireplace_qu'].fillna('NoFireplace')

In [ ]:
# cross checked ordinally correct for nofireplace
sns.boxplot(
    x = 'fireplace_qu',
    y = 'sale_price',
    data = df,
    order = ['Ex', 'Gd', 'TA', 'Fa', 'Po','NoFireplace'],
)

**Can see alot of outliers in Gd, TA, and Po (after adding NAN) inside. Hence, this might not be good to insert into our model**

In [ ]:
sns.pairplot(
    y_vars = 'sale_price',
    x_vars = ['garage_type',
    'garage_yr_blt', 'garage_finish', 'garag_cars', 'garage_area',
    'garage_qual', 'garage_cond','sale_price'],
    data = df
)

In [ ]:
# fill in None for now, might drop categorical columns if need to refine model

df['garage_type'] = df['garage_type'].fillna('NoGarage')
df['garage_finish'] = df['garage_finish'].fillna('NoGarage')
df['garage_qual'] = df['garage_qual'].fillna('NoGarage')
df['garage_cond'] = df['garage_cond'].fillna('NoGarage')

df['garage_area'].fillna(df['garage_area'].mean(),inplace = True)
df['garag_cars'].fillna(df['garag_cars'].mode()[0],inplace = True)
# drop garage year built as null values cannot be replaced, it cannot be replaced by any random values or mean too. 

df.drop(columns = ['garage_yr_blt'], inplace = True)

In [ ]:
# noticed when pool area ==0, other categories also null 

df[df['pool_area'] == 0]

In [ ]:
# too little points to do correlation for pool area 

sns.pairplot(
    x_vars = ['pool_area', 'pool_qc', 'fence', 'misc_feature', 'sale_price'],
    y_vars = 'sale_price',
    data = df
)

In [ ]:
# drop all pool columns 

drop_pool_list = ['pool_qc', 'fence', 'misc_feature']

df = df.drop(columns = drop_pool_list, axis = 1)

In [ ]:
# seeing decreasing trend, and exceptionally high cost built recently 
sns.scatterplot(
    x = 'year_built',
    y = 'sale_price',
    data = df
)

In [ ]:
sns.histplot(x = 'sale_price', data = df)

In [ ]:
df['sale_price_classified'] = df['sale_price'].apply(lambda x: 2 if x > 450000 else 1 if ((x < 450000) & (x > 300000)) else 0)

In [ ]:
df.columns

In [ ]:
for var in df.columns:
    plt.figure(figsize=(15,6))
    plt.subplot(1, 2, 1)
    fig = sns.histplot(x = var, data = df, hue = 'sale_price_classified', color = 'tab10')
    fig.set_title('')
    fig.set_xlabel(var)
    fig.set_ylabel('Counts')

    plt.show()

In [ ]:
['exter_qual',
 'kitchen_qual',
 'bsmt_qual',
 'garage_finish',
 'fireplace_qu',
 'bsmtfin_type1',
 'heating_qc',
 'bsmt_exposure',
 'overall_cond',
 'lot_shape']

**Variables not to use/may not use*

id, pid --> no relevance 

ms_subclass, ms_zoning, cond1, cond2, bldg_type,house_style, overall_cond, ext_1st, ext_2nd, mas_vnr_type,mas_vnr_area
bsmt_cond, bsmt_exposure, bsmt_unf_sf, 2nd_Flor_sf, low_qual_fin_sf, bsmt_full_bath,  bsmt_half_bath, full_bath, half_bath, 
bedroom_abv_grd, kitchen_abv_grd , open_porch_sf, wood_deck_sf, 

--> overlaps with higher sale price, nt in order, hence not good 

pave, no alley, lot shape, land_cotour,util, lot_config, land_slope,roof_style, roof_matl,exter_cond,foundation, 
bsmtfin_type1, bsmt_type2, bsmt_sf2, heating, central_air, electrical, paved_drive, 
->> majority classified under low sale price, which may skew predicted price lower 

bsmtfin_sf1 --> good distribution against sale price clsuter, but having cluster even at near zero with high sale price
year_sold --> not much 


**Variables to use**

1) lot_front -> distribution is clear against sale price

2) lot_area -> distribution is clear against sale price, but need to take care of above 20000

3) Neighbourhood --> can see clusters of higher cost housing in certain areas

4) Overall_qual --> positively correlated to ordinal cat against sale price

5) year_built --> the latest it was built in, the higher the price, need to watch for cluster after year 2000

6) year_removd_add --> the latest it was built in, the higher the price, need to watch for cluster after year 2000

7) exter_qual --> positively correlated to ordinal cat against sale price, cluster in EX

8) bsmt_qual --> positively correlated to ordinal cat against sale price, cluster in EX

9) heating_qc --> most high end sale price are in ex, need watch for cluster at ex with normal sale price. 
              --> can consider interaction terms with other quals to be more accurate for higher selling price
              
              
10) total_bsmt_sf --> distribution is clear against sale price, need to create categorical separate by 3 sale groups for interactions 

11) 1st_flor_sf -->  distribution is clear against sale price, need to create categorical separate by 3 sale groups for interactions 

12) gv_liv_area-->  distribution is clear against sale price, need to create categorical separate by 3 sale groups for interactions 

13) kitchen_qual -->  distribution is clear against sale price, need to create categorical separate by 3 sale groups for interactions 

14) garage_finish --> ordinal cat matches with sales prices. 

15) fireplace_qu -> ordinal cat matches with sales prices.

In [ ]:
plt.figure(figsize=(10,10))
sns.histplot(x = 'lot_area', data = df, hue = 'sale_price_classified', bins =500)

plt.xlim(1000,60000)

In [ ]:
df.groupby('sale_price_classified')['lot_area'].mean()

**Will need to do iteraction term with lot area classified into 3 main cat and interact with lot area it self**

In [ ]:
plt.figure(figsize=(10,10))
sns.histplot(x = 'total_bsmt_sf', data = df, hue = 'sale_price_classified', bins =200)

plt.xlim(1000,3000)

In [ ]:
df.groupby('sale_price_classified')['total_bsmt_sf'].count()

In [ ]:
plt.figure(figsize=(30,10))
sns.histplot(x = 'neighborhood', data = df, hue = 'sale_price_classified')

# plt.xlim(1000,3000)

**comments**
1) Northridge Heights, Stone Brook and Northridge has majority high sale price housing


In [ ]:
=

### Filter out numerical, and categorical (Ordinal) features 

In [ ]:
categorical_features = [col for col in df.columns if df[col].dtype == 'O']

In [ ]:
numerical_features = [col for col in df.columns if df[col].dtype != 'O']

In [ ]:
ordinal_cat = ['lot_shape','util','land_slope','exter_qual','exter_cond','bsmt_qual','bsmt_cond','bsmt_exposure',
               'bsmtfin_type1','bsmt_type2','electrical','kitchen_qual','functional','fireplace_qu','garage_finish',
               'garage_qual','garage_cond','paved_drive','overall_cond','heating_qc'
]

In [ ]:
for var in ordinal_cat:
    plt.figure(figsize=(15,6))
    plt.subplot(1, 2, 1)
    fig = sns.boxplot(x = var, y = 'sale_price', data = df)
    fig.set_title('')
    fig.set_xlabel(var)
    fig.set_ylabel('Sale_price')

    plt.show()

## Pre - processing

**Converting categorical variables**

In [ ]:
alist = []
ordinal_score_list = []
for col in ordinal_cat:
    # instantiate 
    alist = []
    # append column 
    alist.append(col)
    # applying OneHotCode
    df_to_OHE = df[[col]]
    # define one hot encoding
    encoder = OneHotEncoder(sparse=False)
    # transform data
    feature_arr = encoder.fit_transform(df_to_OHE)
    
    feature_labels = encoder.categories_
    feature_labels = np.array(feature_labels).ravel()
    
    feature_encoded = pd.DataFrame(feature_arr, columns = feature_labels )

    # assign X and y

    X = feature_encoded
    y = df['sale_price']

    # Create train/test splits.
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2,
        random_state = 123
    )

    # Scale our data.
    # Relabeling scaled data as "Z" is common.
    sc = StandardScaler()
    # Z_train = sc.fit_transform(X_train)
    sc.fit(X_train) # fit should only see train data, 
                    #else if put in test, will have leak from the train data 
    X_train = sc.transform(X_train)
    X_test = sc.transform(X_test)

    lr = LinearRegression()

    # cross val mse
    cross_score = - cross_val_score(
        lr,
        X_train,
        y_train,
        cv = 3,
        scoring = 'neg_mean_squared_error'
    ).mean()
    # append cross score
    alist.append(cross_score)
    
    ordinal_score_list.append(alist)


ordinal_score_list = pd.DataFrame(ordinal_score_list)                   
ordinal_score_list = ordinal_score_list.rename(columns = {0: 'Column', 1: 'Score'})
ordinal_score_list = ordinal_score_list.sort_values(by = 'Score', ascending = True)     

In [ ]:
# picking top 5 ordinal categorical columns for model fitting

ordinal_score_list.head(10)

In [ ]:
df['bsmt_full_bath'].unique()

In [ ]:
# assign 0 to bsmt_full_bath since no basement 
df.loc[df['bsmt_full_bath'].isnull(), 'bsmt_full_bath'] = 0

In [ ]:
# assign 0 to bsmt_full_bath since no basement 
df.loc[df['bsmt_half_bath'].isnull(), 'bsmt_half_bath'] = 0

In [ ]:
# make copy of main df as Backup

main_df = df.copy()

**numerical feature selection**

In [ ]:
variables = ['overall_qual', 'gr_liv_area', 'garage_area',
       'garag_cars', '1st_flr_sf', 'total_bsmt_sf', 'year_built', 'full_bath',
       'year_remod_add', 'tot_rms_abv_grd', 'mas_vnr_area', 'fireplace',
       'bsmtfin_sf1', 'lot_front', 'open_porch_sf', 'wood_deck_sf', 'lot_area']

variables_1 = ['overall_qual', 'gr_liv_area', 'garage_area',
       'garag_cars', '1st_flr_sf', 'total_bsmt_sf', 'year_built', 'full_bath',
       'year_remod_add', 'tot_rms_abv_grd', 'mas_vnr_area', 'fireplace',
       'bsmtfin_sf1', 'lot_front', 'open_porch_sf', 'wood_deck_sf', 'lot_area',
       'half_bath', 'bsmt_full_bath', '2nd_flr_sf', 'bsmt_unf_sf',
       'screen_porch', 'bedroom_abv_gr', '3ssn_porch', 'mo_sold', 'pool_area',
       'bsmt_sf2', 'misc_val', 'yr_sold', 'low_qual_fin_sf','bsmt_half_bath', 'ms_subclass', 'kitchen_abv_gr', 'overall_cond',
       'enclosed_porch']

variables_2 = ['overall_qual','gr_liv_area', 'garage_area',
       'garag_cars', '1st_flr_sf', 'total_bsmt_sf', 'full_bath',
        'tot_rms_abv_grd', 'mas_vnr_area', 'fireplace',
       'bsmtfin_sf1','year_remod_add','year_built', 'lot_front']

variables_3 = ['overall_qual', 'gr_liv_area', 'garage_area',]

In [ ]:
num_cat = df[variables]

**Ordinal Feature Selection**

In [ ]:
# choose categorical columns 

columns = list(ordinal_score_list.head(10)['Column'].values)
columns

In [ ]:
ord_cat = 0
#increment by one for every field, used to differentiate between first iteration vs the rest
i = 0

for field in columns:
    # assign df1 as temp df -->> and dummify it, drop first column too 
 
    df1 = pd.get_dummies(data = df[field], drop_first = True, columns = field, prefix = field)
    
    # if first iter, assign df final to copied of df1
    if i == 0:
        ord_cat = df1.copy()
    # not first iter, concat with df_final to stack with dummified df1
    else:
         ord_cat = pd.concat([ord_cat, df1], axis = 1) 
            
    i += 1

ord_cat

In [ ]:
# concat for model variables 

model_feat = pd.concat([num_cat,ord_cat], axis =1 )
model_feat

In [ ]:
model_feat = pd.concat([model_feat,df['sale_price']], axis = 1)

### Polynomial features

In [ ]:
poly_feat = ['overall_qual','gr_liv_area', 'garage_area',
       'garag_cars', '1st_flr_sf', 'total_bsmt_sf', 'full_bath',
        'tot_rms_abv_grd', 'mas_vnr_area', 'fireplace',
       'bsmtfin_sf1','year_remod_add','year_built', 'lot_front']

In [ ]:
model_feat.columns

In [ ]:
empty_list = []
df_combi_mse = []
for val in list(combinations(model_feat.columns.drop(['sale_price','above300K_1']), 3)):
    # instantiate empty list 
    empty_list = []

    # append in combination
    empty_list.append(list(val))

    X = model_feat[list(val)]
    y = model_feat['sale_price']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= np.random.seed(123))

    # instantiate lr
    lr = LinearRegression()

    #append in cross score
    empty_list.append(-cross_val_score(lr, X_train, y_train, cv=3, scoring ='neg_mean_squared_error').mean())   


    df_combi_mse.append(empty_list)
        
df_combi_mse = pd.DataFrame(df_combi_mse)
df_combi_mse = df_combi_mse.rename(columns = {0: 'Combinations', 1: 'Score'})
df_combi_mse = df_combi_mse.sort_values(by = 'Score', ascending = True)
df_combi_mse

In [ ]:
filter_best_combi = list(df_combi_mse['Combinations'].head(5))

In [ ]:
filter_best_combi = [' '.join(level) for level in filter_best_combi] 
filter_best_combi

In [ ]:
X = df[poly_feat]
poly = PolynomialFeatures(include_bias=False, degree = 2)


In [ ]:
poly.fit(X)

In [ ]:
X_poly = poly.fit_transform(X)

In [ ]:
X_poly

In [ ]:
poly_concat = pd.DataFrame(X_poly, columns=poly.get_feature_names(poly_feat))[filter_best_combi]

In [ ]:
poly_concat

In [ ]:
model_feat = pd.concat([model_feat,poly_concat], axis = 1)

In [ ]:
model_feat

**Log value**

In [ ]:
sns.pairplot(
    x_vars = variables,
    y_vars = 'sale_price',
    data = df

)

In [ ]:
log_variables = ['year_built', 'year_remod_add', 'gr_liv_area', 'garage_area']

In [ ]:
for feat in log_variables:
    data = model_feat.copy()
    if 0 in data[feat].unique():
        pass
    else:
        plt.figure(figsize=(15,6))
        plt.subplot(1, 2, 1)
        data[feat] = np.log(data[feat])
        data['sale_price'] = np.log(data['sale_price'])
        fig = sns.regplot(x = feat, y = 'sale_price', data = data)
        fig.set_xlabel(feat)
        fig.set_ylabel('Sale price')

In [ ]:
sns.histplot(
     x = 'sale_price',
    data = data

)

**Apply Standard Scaler**

In [ ]:
# assign X and y

X = df[variables_1]
y = df['sale_price']


# Create train/test splits.
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state = 123
)

# Scale our data.
# Relabeling scaled data as "Z" is common.
sc = StandardScaler()
# Z_train = sc.fit_transform(X_train)
sc.fit(X_train) # fit should only see train data, 
                #else if put in test, will have leak from the train data 
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

## Modelling

**Linear Regression - First pass**

In [ ]:
lr = LinearRegression()

lr.fit(X_train,y_train)

# cross val mse
cross_score = - cross_val_score(
    lr,
    X_train,
    y_train,
    cv = 3,
    scoring = 'neg_mean_squared_error'
).mean()

# test mse
mse_test = mean_squared_error(
    y_test,
    lr.predict(X_test)
)
# cross - test 
mse_train_minus_test = cross_score - mse_test

if mse_train_minus_test > 0:
    print(f'MSE for train: {cross_score}')
    print(f'MSE for test: {mse_test}')
    print(f'Underfited by: {mse_train_minus_test}')
else:
    print(f'MSE for train: {cross_score}')
    print(f'MSE for test: {mse_test}')
    print(f'Overfitted by: {mse_train_minus_test}')
print(f'RMSE: {mse_test**0.5}')

In [ ]:
# noticed at higher ends of the sale_price,
# model is calculating below real sales_price

ax = sns.jointplot(
    x = lr.predict(X_test),
    y = y_test,
    kind = 'reg'
)
ax.ax_joint.set_xlabel('Predicted Sale Price')
ax.ax_joint.set_ylabel('Actual Sale Price')

**Lasso Regression**

In [ ]:
# initiate 
lasso = LassoCV(
    cv=5,
    max_iter=50000,
    n_alphas=200,
)

# Fit model using best ridge alpha!
lasso.fit(X_train, y_train);


lasso.alpha_

In [ ]:
lasso_best = Lasso(alpha = lasso.alpha_)

In [ ]:
# fit lasso best 

lasso_best.fit(X_train,y_train)

In [ ]:
# cross val mse
cross_score = - cross_val_score(
    lasso_best,
    X_train,
    y_train,
    cv = 3,
    scoring = 'neg_mean_squared_error'
).mean()

# test mse
mse_test = mean_squared_error(
    y_test,
    lasso_best.predict(X_test)
)
# cross - test 
mse_train_minus_test = cross_score - mse_test

if mse_train_minus_test > 0:
    print(f'MSE for train: {cross_score}')
    print(f'MSE for test: {mse_test}')
    print(f'Underfited by: {mse_train_minus_test}')
else:
    print(f'MSE for train: {cross_score}')
    print(f'MSE for test: {mse_test}')
    print(f'Overfitted by: {mse_train_minus_test}')
print(f'RMSE: {mse_test**0.5}')

In [ ]:
# MSE for test: 1498556570.0823781
# MSE for test: 1292841509.1969898
# Underfited by: 205715060.88538837
# RMSE: 14342.770335098738

In [ ]:
# noticed at higher ends of the sale_price,
# model is calculating below real sales_price

ax = sns.jointplot(
    x = lasso_best.predict(X_test),
    y = y_test,
    kind = 'reg'
)
ax.ax_joint.set_xlabel('Predicted Sale Price')
ax.ax_joint.set_ylabel('Actual Sale Price')

In [ ]:
lasso_best.predict(X_test).max()

In [ ]:
# find out index 339 having anomaly 
anomaly = {num: val for num,val in enumerate (lasso_best.predict(X_test)>638000) if val == True}
anomaly

In [ ]:
y_test.values[339]

In [ ]:
# find out other similar prices with 160000
{num: val for num,val in enumerate (y_test == y_test.values[339]) if val == True}

In [ ]:
Ab_val= pd.DataFrame(X_test[339])
val1= pd.DataFrame(X_test[31])
val2= pd.DataFrame(X_test[286])

In [ ]:
Ab_val.shape

In [ ]:
val1.shape

In [ ]:
val2.shape

In [ ]:
sample = pd.concat([Ab_val,val1,val2], axis = 1)
sample.columns = ['Ab', 'norm1', 'norm2']

In [ ]:
# True outlier compared to two other rows with similar coefficients against sale price

sample.sort_values(by = 'Ab', ascending = False)

In [ ]:
=

In [ ]:
df.drop(index = 960, inplace = True)
df.reset_index(inplace = True)

In [ ]:
# assign X and y

X = df[variables_1]
y = df['sale_price']


# Create train/test splits.
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state = 123
)

# Scale our data.
# Relabeling scaled data as "Z" is common.
sc = StandardScaler()
# Z_train = sc.fit_transform(X_train)
sc.fit(X_train) # fit should only see train data, 
                #else if put in test, will have leak from the train data 
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# initiate 

lasso = LassoCV(
    cv=5,
    max_iter=50000,
    n_alphas=200,
)
# Fit model using best ridge alpha!
lasso.fit(X_train, y_train)
lasso_best = Lasso(alpha = lasso.alpha_)


# fit lasso best with variables 
lasso_best.fit(X_train, y_train)

# cross val mse
cross_score = - cross_val_score(
    lasso_best,
    X_train,
    y_train,
    cv = 3,
    scoring = 'neg_mean_squared_error'
).mean()

# test mse
mse_test = mean_squared_error(
    y_test,
    lasso_best.predict(X_test)
)
# cross - test 
mse_train_minus_test = cross_score - mse_test

if mse_train_minus_test > 0:
    print(f'MSE for train: {cross_score}')
    print(f'MSE for test: {mse_test}')
    print(f'Underfited by: {mse_train_minus_test}')
else:
    print(f'MSE for train: {cross_score}')
    print(f'MSE for test: {mse_test}')
    print(f'Overfitted by: {mse_train_minus_test}')
print(f'RMSE: {mse_test**0.5}')

In [ ]:
# noticed at higher ends of the sale_price,
# model is calculating below real sales_price

ax2 = sns.jointplot(
    x = lasso_best.predict(X_test),
    y = y_test,
    kind = 'reg'
)
ax2.ax_joint.set_xlabel('Predicted Sale Price - No log')
ax2.ax_joint.set_ylabel('Actual Sale Price')

In [ ]:
# hwo to plot the series 
plt.figure(figsize=(10,10))
pd.Series(lasso_best.coef_, index = df[variables_1].columns).sort_values(ascending = False).plot.bar()

In [ ]:
data.columns.drop(['sale_price', 'above300K_1'])

In [ ]:
# hwo to plot the series 
plt.figure(figsize=(10,10))
pd.Series(lasso_best.coef_, index = data.columns.drop(['sale_price', 'above300K_1'])).sort_values(ascending = False).plot.bar()

In [ ]:
grouped = df.groupby('neighborhood')['sale_price']

In [ ]:
feature_set_model = SelectFromModel(lasso_best)
feature_set_model.fit(X_train,y_train)

In [ ]:
len(X_train)

In [ ]:
len(y_train)

In [ ]:
feature_set_model.get_support()

In [ ]:
#refined model 
refined = X.columns[feature_set_model.get_support()]

In [ ]:
X = model_feat[refined]
y = model_feat['sale_price']

# Create train/test splits.
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state = 123
)

# Scale our data.
# Relabeling scaled data as "Z" is common.
sc = StandardScaler()
# Z_train = sc.fit_transform(X_train)
sc.fit(X_train) # fit should only see train data, 
                #else if put in test, will have leak from the train data 
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# fit 

lasso.fit(X_train,y_train)

lasso.alpha_

In [ ]:
# create lasso best 2 

lasso_best_2 = Lasso(alpha=lasso.alpha_)

In [ ]:
lasso_best_2.fit(X_train,y_train)

In [ ]:
# cross val mse
cross_score = - cross_val_score(
    lasso_best_2,
    X_train,
    y_train,
    cv = 3,
    scoring = 'neg_mean_squared_error'
).mean()

# test mse
mse_test = mean_squared_error(
    y_test,
    lasso_best_2.predict(X_test)
)
# cross - test 
mse_train_minus_test = cross_score - mse_test

if mse_train_minus_test > 0:
    print(f'MSE for train: {cross_score}')
    print(f'MSE for test: {mse_test}')
    print(f'Underfited by: {mse_train_minus_test}')
else:
    print(f'MSE for train: {cross_score}')
    print(f'MSE for test: {mse_test}')
    print(f'Overfitted by: {mse_train_minus_test}')
print(f'RMSE: {mse_test**0.5}')

In [ ]:
# noticed at higher ends of the sale_price,
# model is calculating below real sales_price

ax2 = sns.jointplot(
    x = lasso_best_2.predict(X_test),
    y = y_test,
    kind = 'reg'
)
ax2.ax_joint.set_xlabel('Predicted Sale Price - No log')
ax2.ax_joint.set_ylabel('Actual Sale Price')

In [ ]:
X.columns

In [ ]:
# noticed at higher ends of the sale_price,
# model is calculating below real sales_price

ax2 = sns.jointplot(
    x = lasso_best_2.predict(X_test),
    y = y_test,
    kind = 'reg'
)
ax2.ax_joint.set_xlabel('Predicted Sale Price - No log')
ax2.ax_joint.set_ylabel('Actual Sale Price')